In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as Layer


from datetime import datetime

import tensorboard
import matplotlib.pyplot as plt

import numpy as np

In [5]:
# load CIFAR Dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

x_train = train_images.reshape(-1, 32, 32, 3)
x_test = test_images.reshape(-1, 32, 32, 3)

**LeNet CNN**

In [8]:
lenet_CNN = tf.keras.models.Sequential()

lenet_CNN.add(Layer.Conv2D(6, (5, 5), padding='same', activation='relu'))
lenet_CNN.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

lenet_CNN.add(Layer.Conv2D(16, (5, 5), padding='same', activation='relu'))
lenet_CNN.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

lenet_CNN.add(Layer.Conv2D(120, (5, 5), padding='same', activation='relu'))

lenet_CNN.add(Layer.Flatten())
lenet_CNN.add(Layer.Dense(84))
lenet_CNN.add(Layer.Activation('relu'))

lenet_CNN.add(Layer.Dense(10))
lenet_CNN.add(Layer.Activation('softmax'))

lenet_CNN.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])

lenet_CNN.build(input_shape=(1,32,32,3))

lenet_CNN.summary()


logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', min_delta=.0001, patience=3, verbose=5,
    mode='auto', baseline=None, restore_best_weights=False
)



# Training the model.
lenet_CNN.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=25,
    callbacks=[tensorboard_callback, early_stopping_callback])


score = lenet_CNN.evaluate(x_test, test_labels)
print('Test accuracy:', score[1])
print('Test loss:', score[0])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (1, 32, 32, 6)            456       
                                                                 
 max_pooling2d_6 (MaxPooling  (1, 16, 16, 6)           0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (1, 16, 16, 16)           2416      
                                                                 
 max_pooling2d_7 (MaxPooling  (1, 8, 8, 16)            0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (1, 8, 8, 120)            48120     
                                                                 
 flatten_3 (Flatten)         (1, 7680)                

**Feedforward network**

In [9]:
feed_forward_nw = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(32, 32, 3)),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(84, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

feed_forward_nw.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

feed_forward_nw.summary()



logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Training the model.
feed_forward_nw.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=25,
    callbacks=[tensorboard_callback])

# Evaluate
score = feed_forward_nw.evaluate(x_test, test_labels)

print('Test accuracy:', score[1])
print('Test loss:', score[0])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 3072)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                 18438     
                                                                 
 dense_9 (Dense)             (None, 16)                112       
                                                                 
 dense_10 (Dense)            (None, 120)               2040      
                                                                 
 dense_11 (Dense)            (None, 84)                10164     
                                                                 
 dense_12 (Dense)            (None, 10)                850       
                                                                 
Total params: 31,604
Trainable params: 31,604
Non-trai